In [1]:
'''
Multidimensional Retail Data Centralization
-------
This script extracts data, cleans and uploads it all to designated tables in sales_data database.
Methods have been predefined so variables are set to organize the extraction, cleaning and uploading process for each set of data.

Contents
-------
    * Imports 
    * Data Extraction Details - assigning urls/links, endpoints and api-keys necessary to run the code.
    * Classes Usage - instance created and initialized and variables assigned to each class.

    The listed cell blocks perform the following for each dataset: Data Extraction, Transformation and Loading to sales_data.
    * Legacy Users
    * Card Data
    * Business Store Data
    * Products Data
    * Orders Table
    * Sales Date Times

'''

'\nMultidimensional Retail Data Centralization\n-------\nThis script extracts data, cleans and uploads it all to designated tables in sales_data database.\nMethods have been predefined so variables are set to organize the extraction, cleaning and uploading process for each set of data.\n\nContents\n-------\n    * Imports \n    * Data Extraction Details - assigning urls/links, endpoints and api-keys necessary to run the code.\n    * Classes Usage - instance created and initialized and variables assigned to each class.\n\n    The listed cell blocks perform the following for each dataset: Data Extraction, Transformation and Loading to sales_data.\n    * Legacy Users\n    * Card Data\n    * Business Store Data\n    * Products Data\n    * Orders Table\n    * Sales Date Times\n\n'

In [2]:
#Imports
from database_utils import DatabaseConnector
from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from data_processing import DataProcessor

In [3]:
#Data Extraction Details
creds_path = '/Users/itsanya/AiCore/MRDC/db_creds.yaml'

pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'

b_store_api_key = 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'
number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}'

s3_address = 's3://data-handling-public/products.csv' 

s3_url = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'

In [4]:
#Classes Usage
db_connector_instance = DatabaseConnector(creds_path) 
data_extractor = DataExtractor(db_connector_instance) 
data_cleaner = DataCleaning()
data_processor = DataProcessor()

In [5]:
#Legacy Users
#Extract
table_name = 'legacy_users'
users_df = data_extractor.read_rds_table(table_name)

#Clean
fully_cleaned_users_df = data_cleaner._clean_users_df(users_df)

fully_cleaned_users_df

#Upload to dim_users
#db_connector_instance._upload_to_db(df=fully_cleaned_users_df, table_name='dim_users')

/Users/itsanya/AiCore/MRDC/data_processing.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(lambda address: str(address).replace('\n', ' ') if pd.notna(address) else address)
/Users/itsanya/AiCore/MRDC/data_processing.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = pd.to_datetime(df[column_name], format='mixed', errors='coerce')
/Users/itsanya/AiCore/MRDC/data_processing.py:116: SettingWithCopyWarning: 
A value is trying to be set on a cop

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,1,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0 59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,2,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a Lynne terrace McCarthymouth TF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,3,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive Joanborough SK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,4,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow New Tracy W22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,5,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass Hunterborough NN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...,...
15279,15280,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I Jones lodge Oliviaborough E8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15280,15281,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway Millsfurt L4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15281,15282,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5 15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15282,15283,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357 Anthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


In [6]:
#Card Data
#Extract
card_df = data_extractor._retrieve_pdf_data(pdf_link)

#Clean
fully_cleaned_card_df = data_cleaner._clean_card_data(card_df)

fully_cleaned_card_df

#Upload to dim_card_details
db_connector_instance._upload_to_db(df=fully_cleaned_card_df, table_name='dim_card_details')

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Data uploaded to 'dim_card_details' table successfully.


In [7]:
#Business Store Data
#Extract
data_extractor.set_api_key(b_store_api_key)
number_of_stores = data_extractor.list_number_of_stores(number_of_stores_endpoint)
b_store_df = data_extractor.retrieve_stores_data(store_endpoint, number_of_stores)

#Clean
fully_cleaned_b_store_df = data_cleaner._clean_b_store_data(b_store_df)

fully_cleaned_b_store_df

#Upload to dim_store_details
#db_connector_instance._upload_to_db(df=fully_cleaned_b_store_df, table_name='dim_store_details')

Number of stores: 451


/Users/itsanya/AiCore/MRDC/data_processing.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=column_names, inplace=True)


,index,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,1,N/A,NaN,N/A,WEB-1388012W,325.0,2010-06-12,Web Portal,NaN,GB,Europe
1,2,"Flat 72W Sally isle East Deantown E7B 8EB, Hig...",51.62907,High Wycombe,HI-9B97EE4E,34.0,1996-10-25,Local,-0.74934,GB,Europe
2,3,"Heckerstraße 4/5 50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92.0,2013-04-12,Super Store,12.16179,DE,Europe
3,4,"5 Harrison tunnel South Lydia WC9 2BE, Westbury",51.26000,Westbury,WE-1DE82CEE,69.0,2014-01-02,Super Store,-2.18750,GB,Europe
4,5,"Studio 6 Stephen landing South Simon B77 2WA, ...",53.02330,Belper,BE-18074576,35.0,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...
340,341,"936 Foster forge Tracyside W0 3GT, Walton-on-t...",51.84819,Walton-on-the-Naze,WA-A41DA979,98.0,2014-06-04,Super Store,1.26738,GB,Europe
341,342,"Flat 7 Stephanie lake Morrisside HP8 8LH, Cowes",50.76306,Cowes,CO-473A9FBB,94.0,2008-06-08,Super Store,-1.29772,GB,Europe
342,343,"Studio 8 Moss mall West Linda M0E 6XR, High Wy...",51.62907,High Wycombe,HI-EEA7AE62,33.0,1998-05-14,Local,-0.74934,GB,Europe
343,344,"Baumplatz 6 80114 Kötzting, Bretten",49.03685,Bretten,BR-662EC74C,35.0,2020-10-17,Local,8.70745,DE,Europe


In [8]:
#Products Data
#Extract
prods_df = data_extractor.extract_from_s3(s3_address)

#Clean
cleaned_prod_w = data_cleaner._convert_product_weights(prods_df)
cleaned_prod_w

fully_cleaned_prods_df = data_cleaner._clean_products_data(cleaned_prod_w)
fully_cleaned_prods_df

#Upload to dim_store_details
#db_connector_instance._upload_to_db(df=fully_cleaned_prods_df, table_name='dim_products')

,index,product_name,product_price,weight(kg),category,EAN,date_added,uuid,product_status,product_code
0,1,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,2,Tiffany's World Day Out At The Park,£12.99,0.48,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,3,Tiffany's World Pups Picnic Playset,£7.00,0.59,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,4,Tiffany's World Wildlife Park Adventures,£12.99,0.54,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,5,Cosatto Cosy Dolls Pram,£30.00,1.91,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
...,...,...,...,...,...,...,...,...,...,...
1841,1842,Goodmans Qi Autosense In Car Phone Holder,£15.00,0.125,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1842,1843,Goodmans Qi Wireless Charging Phone Stand,£15.00,0.134,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1843,1844,RAC 12V Wet & Dry Vacuum,£18.00,0.82,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b
1844,1845,Goodmans Qi Autosense In Car Phone Holder,£15.00,0.125,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_avaliable,A3-7619070S


In [9]:
#Orders Table
#Extract
table_name = 'orders_table'
orders_df = data_extractor.read_rds_table(table_name)

#Clean
fully_cleaned_orders_df = data_cleaner._clean_orders_df(orders_df)
fully_cleaned_orders_df

#Upload to orders_table
#db_connector_instance._upload_to_db(df=fully_cleaned_orders_df, table_name='orders_table')

,index,date_uuid,user_uuid,card_number,store_code,product_code,product_quantity
0,1,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,3
1,2,0423a395-a04d-4e4a-bd0f-d237cbd5a295,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,2
2,3,65187294-bb16-4519-adc0-787bbe423970,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,2
3,4,579e21f7-13cb-436b-83ad-33687a4eb337,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,2
4,5,00ab86c3-2039-4674-b9c1-adbcbbf525bd,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,2
...,...,...,...,...,...,...,...
120118,120119,f0e8fff6-9998-4661-954b-0e258e09d33c,95c74b0a-d495-4359-b1c0-e2da511e8403,575421945446,KA-FA7ED3B8,C9-6827622o,4
120119,120120,1c80940a-d186-4ba9-9daa-8abd1aceae32,5d6fa6fe-e583-4baf-8bbb-d1dd6e2b551f,4971858637664481,WA-A41DA979,I0-1146408B,1
120120,120121,58598aca-049c-418e-8e39-46327634a7f1,48b7f1fc-db13-4611-ad8e-3dac0b759488,4971858637664481,WEB-1388012W,A4-5443400b,4
120121,120122,3a76f661-0707-4fbc-9862-f21d3249f581,51c0b538-7ded-4697-8e84-9f7aa13f9112,4971858637664481,SO-6D328417,E9-2782979e,4


In [10]:
#Sales Date Times
#Extract
sdt_df = data_extractor.extract_sdt(s3_url)

#Clean
fully_cleaned_sdt_df = data_cleaner._clean_sdt_df(sdt_df)
fully_cleaned_sdt_df

#Upload to dim_date_times
#db_connector_instance._upload_to_db(df=fully_cleaned_sdt_df, table_name='dim_date_times')

/Users/itsanya/AiCore/MRDC/data_processing.py:99: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sdt_df['timestamp'] = pd.to_datetime(sdt_df['timestamp'], errors='coerce').dt.time


,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
...,...,...,...,...,...,...
120156,22:56:56,11,2022,12,Evening,d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7
120157,18:25:20,5,1997,31,Evening,f7722027-1aae-49c3-8f8d-853e93f9f3e6
120158,18:21:40,9,2011,13,Evening,4a3b9851-52e1-463c-ac81-1960f141444e
120159,19:10:53,7,2013,12,Evening,64974909-0d4b-42a2-822a-73b5695e8bfb
